# CUDA-BEVFusion Evaluation on nuScenes Dataset

This notebook runs CUDA-BEVFusion inference on the nuScenes validation set and computes accuracy metrics (mAP and NDS).

## Overview
- **Model**: ResNet50 BEVFusion-Base
- **Dataset**: nuScenes validation set
- **Metrics**: mAP (mean Average Precision) and NDS (nuScenes Detection Score)


In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Compute capability: {torch.cuda.get_device_capability(0)}")


## Step 1: Install Dependencies


In [ ]:
# Install system dependencies
import os
os.system("apt-get update -qq")
os.system("apt-get install -y -qq libprotobuf-dev protobuf-compiler cmake build-essential git")

# Install Python dependencies
os.system("pip install -q onnx==1.12.0 onnx_simplifier==0.4.8 onnxsim==0.4.10 onnxruntime==1.13.1")
os.system("pip install -q --extra-index-url https://pypi.ngc.nvidia.com onnx-graphsurgeon pytorch-quantization")
os.system("pip install -q shapely==1.8.0 pyyaml")
os.system("pip install -q mmcv-full mmdet mmdet3d nuscenes-devkit")
os.system("pip install -q torchpack")


## Step 2: Clone CUDA-BEVFusion Repository


In [ ]:
import os
import sys

# Clone repository (adjust path if needed)
repo_path = "/content/CUDA-BEVFusion"
if not os.path.exists(repo_path):
    os.system(f"git clone --recursive https://github.com/NVIDIA-AI-IOT/CUDA-BEVFusion.git {repo_path}")
else:
    print(f"Repository already exists at {repo_path}")

os.chdir(repo_path)
print(f"Current directory: {os.getcwd()}")


## Step 3: Download Models and Dataset

**Note**: You'll need to download:
1. **model.zip** from [NVBox](https://nvidia.box.com/shared/static/vc1ezra9kw7gu7wg3v8cwuiqshwr8b39) or [Baidu Drive](https://pan.baidu.com/s/1BiAoQ8L7nC45vEwkN3bSGQ?pwd=8jb6)
2. **nuScenes dataset** - You need the full nuScenes validation set with annotations

For Colab, you can upload these files or use gdown to download from Google Drive.


In [ ]:
# Install gdown for downloading from Google Drive
os.system("pip install -q gdown")

# Create directories
os.makedirs("model", exist_ok=True)
os.makedirs("data/nuscenes", exist_ok=True)

print("Please download the following files:")
print("1. model.zip - Extract to ./model/")
print("2. nuScenes dataset - Place in ./data/nuscenes/")
print("\nYou can use the following commands:")
print("  - Upload files via Colab file browser")
print("  - Or use gdown if you have Google Drive links")
print("\nExample:")
print("  os.system('gdown <file_id> -O model.zip')")
print("  os.system('unzip -q model.zip -d model/')")


## Step 4: Setup Environment Variables


In [ ]:
import os

# Detect CUDA version
import subprocess
result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
cuda_version = "12.6"  # Default, adjust based on your system
if "release 11" in result.stdout:
    cuda_version = "11.4"
elif "release 12" in result.stdout:
    cuda_version = "12.6"

# Set environment variables
os.environ["DEBUG_MODEL"] = "resnet50"
os.environ["DEBUG_PRECISION"] = "fp16"
os.environ["USE_Python"] = "ON"
os.environ["SPCONV_CUDA_VERSION"] = cuda_version

# Detect compute capability
if torch.cuda.is_available():
    compute_cap = torch.cuda.get_device_capability(0)
    os.environ["CUDASM"] = f"{compute_cap[0]}{compute_cap[1]}"
    print(f"Compute capability: SM {compute_cap[0]}.{compute_cap[1]}")
else:
    os.environ["CUDASM"] = "80"  # Default to SM 8.0

# CUDA paths (adjust for Colab)
os.environ["CUDA_HOME"] = "/usr/local/cuda"
os.environ["CUDA_Inc"] = f"{os.environ['CUDA_HOME']}/include"
os.environ["CUDA_Lib"] = f"{os.environ['CUDA_HOME']}/lib64"

# TensorRT paths (adjust for Colab - may need to install TensorRT)
# For Colab, TensorRT might be in /usr/lib/x86_64-linux-gnu or needs installation
os.environ["TensorRT_Inc"] = "/usr/include/x86_64-linux-gnu"
os.environ["TensorRT_Lib"] = "/usr/lib/x86_64-linux-gnu"
os.environ["TensorRT_Bin"] = "/usr/bin"

print(f"Model: {os.environ['DEBUG_MODEL']}")
print(f"Precision: {os.environ['DEBUG_PRECISION']}")
print(f"CUDA Version: {os.environ['SPCONV_CUDA_VERSION']}")


## Step 5: Build TensorRT Engines

Build the TensorRT engines from ONNX models.


In [ ]:
# Check if TensorRT engines already exist
model_path = f"model/{os.environ['DEBUG_MODEL']}/build"
if os.path.exists(model_path) and os.path.exists(f"{model_path}/camera.backbone.plan"):
    print(f"TensorRT engines already exist at {model_path}")
    print("Skipping build step. If you want to rebuild, delete the build directory.")
else:
    print("Building TensorRT engines...")
    print("Note: This requires TensorRT to be installed and may take several minutes.")
    # Run build script
    os.system("bash tool/build_trt_engine.sh")


## Step 6: Build Python Library

Build the libpybev.so Python extension.


In [ ]:
# Generate protobuf code
os.system("bash src/onnx/make_pb.sh")

# Build the library
print("Building Python library...")
os.system("bash tool/run.sh")

# Verify build
build_path = "build"
if os.path.exists(f"{build_path}/libpybev.so"):
    print(f"✓ Successfully built libpybev.so")
    # Add to Python path
    sys.path.insert(0, os.path.abspath(build_path))
    sys.path.insert(0, os.path.abspath("tool"))
    sys.path.insert(0, os.path.abspath("src/common"))
else:
    print("✗ Build failed - libpybev.so not found")


## Step 7: Prepare nuScenes Dataset

Ensure the nuScenes dataset is properly formatted with validation annotations.


In [ ]:
# Check nuScenes dataset structure
nuscenes_root = "data/nuscenes"
required_files = [
    "nuscenes_infos_val.pkl",
    "samples",
    "sweeps",
    "maps"
]

print("Checking nuScenes dataset structure...")
for item in required_files:
    path = os.path.join(nuscenes_root, item)
    if os.path.exists(path):
        print(f"✓ Found: {item}")
    else:
        print(f"✗ Missing: {item}")

# If dataset converter is needed
if not os.path.exists(os.path.join(nuscenes_root, "nuscenes_infos_val.pkl")):
    print("\nNote: You may need to run the dataset converter:")
    print("  python bevfusion/tools/create_data.py nuscenes --root-path data/nuscenes --out-dir data/nuscenes --version v1.0-mini")
    print("\nFor full validation set, use v1.0-trainval")


## Step 8: Run Evaluation on nuScenes Validation Set

This will run inference on all validation samples and compute mAP and NDS metrics.


In [ ]:
# Set up paths for evaluation
config_path = "bevfusion/configs/nuscenes/det/transfusion/secfpn/camera+lidar/resnet50/convfuser.yaml"

# If config doesn't exist, try alternative path
if not os.path.exists(config_path):
    config_path = "configs/nuscenes/det/transfusion/secfpn/camera+lidar/resnet50/convfuser.yaml"

if not os.path.exists(config_path):
    print(f"Config file not found at {config_path}")
    print("Please ensure the config file exists or update the path")
else:
    print(f"Using config: {config_path}")

# Update dataset root in config if needed
try:
    import yaml
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print(f"Dataset root in config: {config.get('dataset_root', 'not set')}")
except ImportError:
    print("PyYAML not installed. Install with: pip install pyyaml")
except Exception as e:
    print(f"Could not read config: {e}")


In [ ]:
# Run evaluation using the test script
# This will compute mAP and NDS on the validation set

import subprocess
import sys

# Change to repository root
os.chdir(repo_path)

# Run evaluation
eval_script = "qat/test-mAP-for-cuda.py"
if os.path.exists(eval_script):
    print("Running evaluation...")
    print("This may take a while depending on dataset size...")
    
    cmd = [
        sys.executable, eval_script,
        "--config", config_path,
        "--eval", "bbox",
        "--out", "results.pkl"
    ]
    
    result = subprocess.run(cmd, capture_output=False, text=True)
    
    if result.returncode == 0:
        print("\n✓ Evaluation completed successfully!")
    else:
        print(f"\n✗ Evaluation failed with return code {result.returncode}")
else:
    print(f"Evaluation script not found at {eval_script}")
    print("Please ensure the script exists in the repository")


## Step 9: Display Results

Display the computed accuracy metrics (mAP and NDS).


In [ ]:
# Check if results file exists and display metrics
results_file = "results.pkl"

if os.path.exists(results_file):
    import pickle
    import json
    
    # Try to load results
    try:
        with open(results_file, 'rb') as f:
            results = pickle.load(f)
        print(f"Loaded results from {results_file}")
        print(f"Number of samples evaluated: {len(results)}")
    except Exception as e:
        print(f"Error loading results: {e}")
    
    # Check for metrics summary
    metrics_file = "metrics_summary.json"
    if os.path.exists(metrics_file):
        with open(metrics_file, 'r') as f:
            metrics = json.load(f)
        
        print("\n" + "="*60)
        print("EVALUATION RESULTS - nuScenes Validation Set")
        print("="*60)
        
        # Display key metrics
        if 'mean_ap' in metrics:
            print(f"\nmAP (mean Average Precision): {metrics['mean_ap']:.4f}")
        if 'nd_score' in metrics:
            print(f"NDS (nuScenes Detection Score): {metrics['nd_score']:.4f}")
        
        # Display per-class metrics if available
        if 'mean_dist_aps' in metrics:
            print("\nPer-class mAP:")
            for class_name, ap in metrics['mean_dist_aps'].items():
                print(f"  {class_name}: {ap:.4f}")
        
        # Display other metrics
        print("\nOther metrics:")
        for key, value in metrics.items():
            if key not in ['mean_ap', 'nd_score', 'mean_dist_aps']:
                if isinstance(value, (int, float)):
                    print(f"  {key}: {value:.4f}")
                else:
                    print(f"  {key}: {value}")
        
        print("="*60)
    else:
        print(f"\nMetrics summary not found. Results file exists but metrics may not have been computed.")
        print("The evaluation script should have generated metrics during evaluation.")
else:
    print(f"Results file not found at {results_file}")
    print("Please run the evaluation step first.")
